# Applied Data Science Capstone Project 
## Week 3

### Read table

In [1]:
#!pip install lxml # uncomment if lxml is not installed

import requests
import pandas as pd

# provide url for scraping 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# grab and read info from html
html = requests.get(url).content
df_list = pd.read_html(html)

In [47]:
# choose which table to look at
# remove "Not assigned" records
df = pd.read_html(html)[0]
df = df[~df.Borough.str.contains("Not assigned")].reset_index(drop = True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [48]:
# print shape of df
df.shape

(103, 3)

### Add geographical coordinates

In [108]:
# get coordinate data
data = pd.read_csv('http://cocl.us/Geospatial_data')
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [112]:
# merge location and coordinate dfs
new_df = pd.merge(df, data, on = "Postal Code")
new_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Cluster and Plot

In [154]:
# !pip install geopy # uncomment if geopy is not installed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# !pip install sklearn # uncomment if sklearn is not installed
from sklearn.cluster import KMeans

# !pip install folium # uncomment if folium is not installed
import folium 

import numpy as np

In [145]:
# select Toronto Neighbourhoods
toronto = new_df[new_df.Borough.str.contains('Toronto')]
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [146]:
# set number of clusters
kclusters = 5

cluster = toronto[["Latitude","Longitude"]]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 0, 0, 3, 0, 2], dtype=int32)

In [147]:
# add clustering labels
toronto.insert(0, 'Cluster Labels', kmeans.labels_)
toronto.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,1,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [152]:
# Toronto coordinates
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(latitude, longitude)

43.6534817 -79.3839347


In [155]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighbourhood'], toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters